<a href="https://colab.research.google.com/github/matteobisardi/ALGOSB_2021_tutorial/blob/main/src/plmDCA_and_evolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Internal to do list: 
- Add general references with papers
- Correct pseudocount for mut spectrum plot
- Add figures from our papers for clarity
- Comment well Evolution part

# 0) General aim of the tutorial
**What we would like to show you**

Evolutionary information can be extracted from Multiple Sequence Alignments via **simple** and **interpretable** models in a matter of minutes. Such models have been used over the years for a variery of biologically relevant tasks [[testo del link]](https://), [[testo del link]](https://), [[testo del link]](https://). 

We will showcase some applications of those models using real data. I. particular we will learn two models using protein sequence family of antibiotic resistant proteins Beta-lactamases ([PFXXX](https://)) and another one ([PFXXX](https://)). 

**What you could do with what you learn**

 - Mutational effect prediction (including multiple mutations)
 - Fast generation of artificial sequences belonging to a target family
 - Explore evolutionary experiments

**Caveats**

The code presented here is a Python implementation of much faster [Julia](https://) code. This means that the models you will learn today in the short time of the presentation are not state-of-the-art. Simple Julia packages are available at [plm and so on](https://).

**Plan of the tutorial**

You can read it off the *Table of Contents* on the left

# 1) Read alignment and preprocess data #

Here we import a **Multiple Sequence Alignment (MSA)**, we compute the weights of the sequences and explore a bit the dataset. 

In [6]:
# import relevant packages
import numpy as np
import warnings
warnings.simplefilter("ignore")

## Import MSA

In [8]:
# import data from GitHub
%%bash
wget -qnc https://raw.githubusercontent.com/matteobisardi/ALGOSB_2021_tutorial/main/data/Beta-lactamase_MSA.fasta

In [41]:
# read data 
data = open('Beta-lactamase_MSA.fasta', 'r')
seqs_temp = data.readlines()
data.close()

# see data
seqs_temp[:10]

['>sp|P62593|BLAT_ECOLX/49-256\n',
 'LNSGKILSFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGPKELTAFLHNMGDHVTRLDRWEPELNAIPNDERDTTMPAAMATTLRKLLTGELLTLAQLIDWMEDKVAGPLLRSALPAGWFIADKSGAERGSRGIIAALGPDGKPSRI--\n',
 '>GBC78419.1/71-315\n',
 'LESGRRVALNGAERFPMASTYKVPIAVQVLTKVDRGELALDQMITLEPADLSPGSGTLSDLRPGALSVRNLLELMLLISDNTATDVLLRVAGAEAVTARLRELGIEEMRVDRPTKAAKRFDEDPRDTATPEAMVALLEQIYRGRALKRELLLDILRCRTGEARLKGLLPPGTEVAHKTGTIGGTTNDVGILTLDAGHVAIA-\n',
 '>4C75_A/27-235\n',
 '-GTGRTIAYRADERFPMCSTFKALAAAAVLAQVDAGKESLDRRITYTKDDLVDYSPVTEKHVGTGMTLAELCEAAITYSDNTAANLLLDEIGPKGLTAFLRSIGDDVTRLDRWEPELNALPGDPRDTTTPAAMAATLRALLLGDALSPAQLTDWMRNTTGDKLIRAGLPAGWRVGDKTGTSYGTRNDIAIIWPNRAPIVLAI\n',
 '>OHB35854.1/62-313\n',
 'LESGELWAFNGSDRFPMMSVFKAPLGAAVLAEVDAGRLDLDETITLTDEDLSPFSPIADAYPGRTYTVGQLLELAVGQSDNTAADVLMRRIGPGVVTAWLRAHRIDDMRVDRYERATLAYMRDPRDTTTPRAALLFLSKLSAGELLSEARLLTLMSTTTGADRLKAGLPEGARLAHKTGTMNPATNDMGIITLDGRRYAVAV\n',
 '>WP_092950155.1/54-261\n',
 '-EAGRQAAYRGDELFPLCSTFKLLLAA

The format of the data is **.fasta**: 
* sequence  *descriptions* start with '**>**' and are followed by <font color = black > sequences </font>  on the next line. 
* seqeunce are *aligned*, they all have the same length


<b>Note:</b> "-" gaps are used to align the data and are treted as any other amino acid in our models. You typically see them at the end of the sequences. 


We only care about sequences, hence we will get rid of descriptions.<br>

In [10]:
# keep only sequences
seqs = [seqs_temp[2*i + 1][:-1] for i in range( int( len(seqs_temp) /2 ) )]

# see data
seqs[:5]

['LNSGKILSFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGPKELTAFLHNMGDHVTRLDRWEPELNAIPNDERDTTMPAAMATTLRKLLTGELLTLAQLIDWMEDKVAGPLLRSALPAGWFIADKSGAERGSRGIIAALGPDGKPSRI--',
 'LESGRRVALNGAERFPMASTYKVPIAVQVLTKVDRGELALDQMITLEPADLSPGSGTLSDLRPGALSVRNLLELMLLISDNTATDVLLRVAGAEAVTARLRELGIEEMRVDRPTKAAKRFDEDPRDTATPEAMVALLEQIYRGRALKRELLLDILRCRTGEARLKGLLPPGTEVAHKTGTIGGTTNDVGILTLDAGHVAIA-',
 '-GTGRTIAYRADERFPMCSTFKALAAAAVLAQVDAGKESLDRRITYTKDDLVDYSPVTEKHVGTGMTLAELCEAAITYSDNTAANLLLDEIGPKGLTAFLRSIGDDVTRLDRWEPELNALPGDPRDTTTPAAMAATLRALLLGDALSPAQLTDWMRNTTGDKLIRAGLPAGWRVGDKTGTSYGTRNDIAIIWPNRAPIVLAI',
 'LESGELWAFNGSDRFPMMSVFKAPLGAAVLAEVDAGRLDLDETITLTDEDLSPFSPIADAYPGRTYTVGQLLELAVGQSDNTAADVLMRRIGPGVVTAWLRAHRIDDMRVDRYERATLAYMRDPRDTTTPRAALLFLSKLSAGELLSEARLLTLMSTTTGADRLKAGLPEGARLAHKTGTMNPATNDMGIITLDGRRYAVAV',
 '-EAGRQAAYRGDELFPLCSTFKLLLAAQVLRRVDQGQERLDRRITYRKVDLVDYSPATAPHAGEGMTVAQLCEAAVTLSDNTAANLLLDSQGPQGLTAWLRSLGDAHTRLDRKEPELNDVPEGERDTTTPRAMARTIWAITHGEALSAAQITDWLVNRTGDKRLRAGMPQ

We then keep only half of the dataset to reduce learning time.

In [11]:
# keep half of the dataset
seqs_small = [seqs[i] for i in range( int( len(seqs)/2)  )]

def subsample_MSA(MSA, )
  return [seqs[i] for i in range( int( len(MSA)/2)  )]

## Convert MSA to matrix

Extract sequence length `L` and number of sequences `M` and convert the _MSA_ in a numerical matrix called `align`.
To do so we need to convert amino acids to numbers.

In [12]:
 # function to convert the amino acids letters into integer numbers
def letter2number(a): 
    switcher = {
        '-': 20,
        'A': 0,
        'C': 1,
        'D':2,
        'E':3,
        'F':4,
        'G':5,
        'H':6,
        'I':7,
        'K':8,
        'L':9,
        'M':10,
        'N':11,
        'P':12,
        'Q':13,
        'R':14,
        'S':15,
        'T':16,
        'V':17,
        'W':18,
        'Y':19,     
    }
    return switcher.get(a,None)


Now we get transform our MSA to a matrix for better numerical handling. 

In [13]:
# get M and L
M = np.size(seqs_small)
L = len(seqs_small[0])

print(f"M = {M}, L = {L} \n")

# convert amino acids chars to numbers
align = np.zeros((M,L)).astype(int)
for m in range (M):
    for i in range (L):
        align[m,i]=letter2number(seqs_small[m][i])  

# see data 
align[:5]

M = 858, L = 202 



array([[ 9, 11, 15, ...,  7, 20, 20],
       [ 9,  3, 15, ...,  7,  0, 20],
       [20,  5, 16, ...,  9,  0,  7],
       [ 9,  3, 15, ..., 17,  0, 17],
       [20,  3,  0, ..., 10, 17, 20]])

We can now put this code in the naive function `fasta2matrix` for future use.

In [14]:
# function to read MSA and convert it to matrix
def fasta2matrix(path):
    data=open(path, 'r')
    seqs_temp = data.readlines()
    data.close()
   
    if seqs_temp[2][0] != '>':
        seqs_temp = [seqs_temp[i][:-1] for i in range(int(len(seqs_temp)))]
        seqs = [ "".join(  seqs_temp[ 4*i +1  :  4*i +4 ]  )  for i in range(int(len(seqs_temp)/4))]
    else:
        seqs = [seqs_temp[2*i+1][:-1] for i in range(int(len(seqs_temp)/2))]
    
    M=np.size(seqs)
    L=len(seqs[0])
    print(f"M = {M}, L = {L} \n")
    align=np.zeros((M,L)).astype(int)
    for m in range(M):
        for i in range (L):
            align[m,i]=letter2number(seqs[m][i])
    return align

In [15]:
# test fasta2matrix
test_MSA = fasta2matrix('Beta-lactamase_MSA.fasta')

M = 1716, L = 202 



## Compute weights

One assumption of our models is that data used to train them should come from independent random samples, this is not the case for protein sequence data.

Biases towards specific sequences are the rule and not the exception (i.e. sequences from model organisms might be overrepresented) and phylogenetic relationship can induce unwanted correlation in the data. <br>For this reason sequences must be reweighted to be able to treat them more as if they were statistically independent. 

The wheight $w_i$ of sequence $s_i$ is given by:

$w_i(\theta) = \left( \#s \,\, | \,\, d_H(s_i, s) > \theta L \, \right)^{-1}$ 

that is the inverse of the number of sequences that are closer to sequence $s_i$ (in terms of amino acid mutations) more than a threshold value $\theta$. Usually $\theta$ is taken as 0.8, that is the threshold is given by $\theta L$ where $L$ is the length of the sequences.

In [16]:
# compute sequence weights
θ = 0.8
msa_compute_weights = np.eye(21)[align.astype(int)]
mat_dist = np.tensordot(msa_compute_weights,msa_compute_weights,[[1,2],[1,2]])/msa_compute_weights.shape[1]
trainingWeights = 1/np.sum(mat_dist>=θ,axis = 1)

Expand the alignment to a binary `(M, 21*L)` array (one-hot encoding) for future use.

In this format each amino acid is represented by a vector of $0$ and $1$ of length $21$. 

E.g:

glycin $\rightarrow$ 'G' $\rightarrow$ 5 $\rightarrow$ [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

In [17]:
# one hot encoder function
def onehot(ali):
    M, L = ali.shape
    q = 21        
    msa = np.zeros( (M,L*q) )
    for m in range (M):
        for i in range (L):
            msa[m, i*q+ali[m,i]]=1 
    return msa

In [18]:
# one hot encode the data
msa = onehot(align)

# see the data
msa

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

`msa` is the version of our alignment with **one-hot encoded** sequences

`align` is the version of our alignment with **amino acids converted to numbers**

## DIY

You can now try to import a MSA you are interested in and convert it to a matrix.

In [ ]:
# import MSA
my_MSA = fasta2matrix('my_data')


# one hot encode
my_MSA_oh = onehot(my_MSA)

# 2) Learn plmDCA model

In [19]:
# import relevant packages
import keras
import time
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

In [20]:
# define useful function
def inputrange(i):
    return [j for j in range(L*q) if j//q != i]
def outputrange(i):
    return [j for j in range(L*q) if j//q == i]

## Learning

We want to learn the parameters of a Potts model, in which the probability of a sequence $ (s_1, s_2,...,s_L)$ is given by 

<font size="3">$p \,(s_1, s_2,...,s_L) = \frac{1}{Z} \exp(-E(s_1, s_2,...,s_L))$</font>. 

The energy is parametrized by couplings $J_{ij}$ (for co-evolution) and by fields $h_i$ (for conservation). We have

<font size="3">$E(s_1, s_2,...,s_L) = - \sum_i h_i(s_i) -\sum_{i,j} J_{ij}(s_i,s_j)$</font>

The learning is done by maximization of the <font color = green >pseudo-likelihood</font>, insted of the full likelihood. The conditional probability of a site $i$ given the rest of the sequence is given by 

<font size="4">$p(s_i | s_{j}, j\ne i) = \frac{\exp(h_i(s_i)+\sum_{j \ne i} J_{ij}(s_i,s_j))}{Z_{s_{j}, j\ne i}}$</font>

We then learn the parameters by maximizing the pseudo log-likelihood (given $M$ sequences in the training set)

<font size="3">$PL = \frac{1}{M}\sum_{m = 1}^{M} \sum_{i = 1}^{L} \log{p(s_i^m | s_{j}^m, j\ne i)}$</font>. 

This pseudo-likelihood maximization is proven to be consistent, that is in the limit of an infinite number of sequences, the parameters are exactly the optimal parameters of the Potts model.

We are going to implement the <font color = green > asymetric </font> version of pseudo-likelihood maximization, that is $J_{ij} \ne J_{ji}$. In this case, each parameter appears in just one conditional probability, and we can optimize each site independently : 

<font size="3">$PL_i = \frac{1}{M}\sum_{m = 1}^{M}  \log{p(s_i^m | s_{j}^m, j\ne i)}$</font>


<font color='red'> **Attention**:  </font> Don't run this  cell again to avoid overwriting the learned parameters. Comment the code if you want to be sure.

In [21]:
# initialize the varibles
q = 21
h = np.zeros(L*q)
J = np.zeros((L*q,L*q))

In [22]:
# define the number of learning epochs
EPO = 100

The learning is done with keras: the input for a given site $i$ (and its conditional probability) is the sequence without the site $i$ in one-hot-encoding, the output is a q-vector corresponding to the probability to emit one of the $q$ amino acids. The learning is done by minimizing the categorigal cross entropy (which is completly equivalent to the maximization of  

$\frac{1}{M}\sum_{m = 1}^{M} \log{p(s_i^m| s_{j}^m,j \ne i)}$.

The parameters that you can change are 
- the number of epochs (number of steps in your gradient descent based algorithm)
- the regularization of the couplings controlled by "kernel_regularizer"
- the regularization of the fields controlled by "bias_regularizer"

The learning should not take around 6 minutes on Colab with GPU Runtime and 10 with CPU. So if you want to save some minutes go on `Runtime` $\rightarrow$ `Change runtime type` $\rightarrow$ `GPU`

*Let's start the learning!* <br>

In [39]:
# start timer
start_t = time.time()

# loop over each site pseudo-likelihood
for i in range(L): 

    model = Sequential()
    model.add(Dense(q, activation="softmax", use_bias = True,input_shape=((L-1)*q,),kernel_regularizer=l2(0.1),bias_regularizer = l2(0.0001)))
    model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])
    batch_size = M
    epochs = EPO
    es = EarlyStopping(monitor='loss',min_delta=0.0001,verbose=0,patience=10)
    history = model.fit(msa[:,inputrange(i)], msa[:,outputrange(i)],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    shuffle=True,
                    callbacks=[es],
                    sample_weight=trainingWeights[:]
                   )

    W=np.asarray(model.get_weights())
    h[i*q:(i+1)*q]=W[1][:]    
    for j in range(i):
        J[j*q:(j+1)*q,i*q:(i+1)*q]=W[0][j*q:(j+1)*q,:]
    for j in range(i+1,L):
        J[j*q:(j+1)*q,i*q:(i+1)*q]=W[0][(j-1)*q:j*q,:]   

    if (i != 0) & (i%10 == 0):
        print(f"Learned {i} sites (out of 202)")
    if i == 101:
        middle_t = time.time()
        print(f"\nWe are half way through! Keep believing! \n{round(middle_t - start_t)} seconds (~ {round((middle_t - start_t)/60)} minutes) have elapsed so far.\n")
        
final_t = time.time()
        
print("\nDone! Enjoy your results.")
print(f"The learning has taken {round(final_t - start_t)} seconds in total (~ {round((final_t - start_t)/60)} minutes).")

Learned 10 sites (out of 202)
Learned 20 sites (out of 202)
Learned 30 sites (out of 202)
Learned 40 sites (out of 202)
Learned 50 sites (out of 202)
Learned 60 sites (out of 202)
Learned 70 sites (out of 202)
Learned 80 sites (out of 202)
Learned 90 sites (out of 202)
Learned 100 sites (out of 202)

We are half way through! Keep believing! 
299 seconds (~ 5 minutes) have elapsed so far.

Learned 110 sites (out of 202)
Learned 120 sites (out of 202)
Learned 130 sites (out of 202)
Learned 140 sites (out of 202)
Learned 150 sites (out of 202)
Learned 160 sites (out of 202)
Learned 170 sites (out of 202)
Learned 180 sites (out of 202)
Learned 190 sites (out of 202)
Learned 200 sites (out of 202)

Done! Enjoy your results.
The learning has taken 591 seconds in total (~ 10 minutes).


## Congrats!
You have now learned your (first?) `plmDCA` model.

The parameters are stored in the variable `h` for the **fields**..

In [ ]:
# fields
h

..and `J` for the **couplings**. 

In [ ]:
# couplings
J

# 3) Predict mutational effects

We can predict the effect of mutations by the ratio of log probabilities between the mutated sequence and the wild type.

"*fitness*" = $log \frac{P(wild-type)}{P(mutant)}$ translates to $E(mutant) - E(wild-type)$.  

If positive the model indicates a higher energy (lower fitness) for the mutant, and viceversa. 

In [ ]:
import scipy.stats as sc
import matplotlib
import matplotlib.pyplot as plt

For single mutations 

$ΔE = E(a_i \rightarrow b_i)$ 

can be computed as
$ΔE = h_i(a_i) - h_i(b_i) + \sum_{j = 1}^L J_{ij}\,(a_j, a_i) - J_{ij}\,(a_j, b_i)$


<b>Note:</b> Rememeber that 

<font size="3">$E(s_1, s_2,...,s_L) = - \sum_i h_i(s_i) -\sum_{i,j} J_{ij}(s_i,s_j)$</font>

that is there is a "-" sign in front of fields $h$ and couplings $J$.

In [ ]:
def get_silico_dms(h,J,seq,L):
    single_muts = np.zeros((L,20))
    for i in range(L):
        tmp = h[i*q+seq[i]]
        for j in range(L):
            tmp += 0.5*(J[j*q+seq[j],i*q+seq[i]]+J[i*q+seq[i],j*q+seq[j]])
        for k in range(20):
            ΔE = tmp - h[i*q+k]
            for j in range(L):
                ΔE -= 0.5*(J[j*q+seq[j],i*q+k]+J[i*q+k,j*q+seq[j]])
            single_muts[i,k] = ΔE
    return single_muts

def get_silico_dms_non_sym(h,J,seq,L):
    single_muts = np.zeros((L,20))
    for i in range(L):
        tmp = h[i*q+seq[i]]
        for j in range(L):
            tmp += J[j*q+seq[j],i*q+seq[i]]
        for k in range(20):
            ΔE = tmp - h[i*q+k]
            for j in range(L):
                ΔE -= J[j*q+seq[j],i*q+k]
            single_muts[i,k] = ΔE
    return single_muts

## plmDCA prediction

Let's compute the change in energy of all single mutants of TEM-1, the first sequence of the alignment. 

In [ ]:
TEM1 = align[0, :]
silico_dms_TEM = get_silico_dms_non_sym(h,J,TEM1,L)

We can compare our prediction with DMS data, that is **Deep Mutational Scannings**. Those are high thoughput experiments were libraries containing all single mutants of a protein are generated and their fitness measured.
A reference can be found here: [Fowler et al.](https://www.nature.com/articles/nmeth.3027)

In [ ]:
dms = np.loadtxt("DMS_TEM1")

Let's compute the spearman correlation.

In [ ]:
sc.spearmanr(dms[:200,:].flatten(),silico_dms_TEM[:200,:].flatten())

Let's plot the data.

In [ ]:
xsize = 3
ysize = 3

vx = silico_dms_TEM[:200,:].flatten()
vy = dms[:200,:].flatten()

multip = 3
sp_val = sc.spearmanr(vx, vy)[0]

fig = plt.figure(figsize = (xsize, ysize), dpi = 200)

pl1 = plt.scatter(vx, vy,s = multip*xsize, alpha = 0.4, linewidth = 0.6, 
    edgecolor = "black", color = "dodgerblue")
plt.tick_params(labelsize = multip*xsize)
plt.yscale("log")
plt.ylim((0.001, 3))
plt.title("Prediction of mutational effects", fontsize = 1.2*multip*xsize)
plt.xlabel("ΔE Potts", fontsize = multip*xsize)
plt.ylabel("Fitness", fontsize = multip*xsize)
plt.text(1.9, 3, f"spearman = {round(sp_val, 2)}", fontsize = 0.8*multip*xsize);

# 4) Simulate experimental evolution 

In [ ]:
from copy import deepcopy
from numpy.random import choice 
import matplotlib
import seaborn as sns

In [ ]:
def cod2amino(a): 
    #to convert the codon 3 letter strings into amino acids (in number format)
    switcher = { 
        "ATA" :  7, "ATC" : 7, "ATT" : 7,  
        "ATG" : 10, 
        "ACA" : 16, "ACC" : 16, "ACG" : 16, "ACT" : 16, 
        "AAC" : 11, "AAT" : 11, 
        "AAA" : 8, "AAG" : 8, 
        "AGC" : 15, "AGT" : 15, "TCA" : 15, "TCC" : 15, "TCG" : 15, "TCT" : 15,
        "AGA" : 14, "AGG" : 14, "CGA" : 14, "CGC" : 14, "CGG" : 14, "CGT" : 14,              
        "CTA" : 9, "CTC" : 9, "CTG" : 9, "CTT" : 9, "TTA" : 9, "TTG" : 9,
        "CCA" : 12, "CCC" : 12, "CCG" : 12, "CCT" : 12, 
        "CAC" : 6, "CAT" : 6, 
        "CAA" : 13, "CAG" : 13, 
        "GTA" : 17, "GTC" : 17, "GTG" : 17, "GTT" : 17, 
        "GCA" : 0, "GCC" : 0, "GCG" : 0, "GCT" : 0, 
        "GAC" : 2, "GAT" : 2, 
        "GAA" : 3, "GAG" : 3, 
        "GGA" : 5, "GGC" : 5, "GGG" : 5, "GGT" : 5, 
        "TTC" : 4, "TTT" : 4,  
        "TAC" : 19, "TAT" : 19, 
        "TAA" : 20, "TAG" :  20, "TGA" :  20, "---"  : 20,
        "TGC" :  1, "TGT" : 1, 
        "TGG" : 18
        }
    return switcher.get(a, None)


def dH(seq1, seq2):
    #hamming distance (number of mutations) between two sequences
    return sum(c1 != c2 for c1, c2 in zip(seq1, seq2))

class Sequence(object):
    #class containing amino acid and DNA sequence
    def __init__(self, amino, codon):
        self.Amino = amino
        self.Codon = codon
        
def mutate_codon(self, new_codon, pos):
    self.Codon[pos] = new_codon

def mutate_amino(self, new_amino, pos):
    self.Amino[pos] = new_amino

def normalize(vec):
    return vec/np.sum(vec)


def cond_prob(position, amino_list, seq, h, J, N, T = 1):
    #conditional probability of amino acid a_i in site i given the rest of the sequence
    proba_vec = np.zeros(len(amino_list))  
    for k, amino in enumerate(amino_list):
        log_proba = h[position*q + amino]
        for i in range(N):
            log_proba += J[i*q + seq[i], position*q + amino]
        proba_vec[k] = np.exp(log_proba/T)
    return normalize(proba_vec)

def cond_prob_sym(position, amino_list, seq, h, J, N, T = 1):
    #conditional probability of amino acid a_i in site i given the rest of the sequence with symmetrized couplings
    proba_vec = np.zeros(len(amino_list))  
    for k, amino in enumerate(amino_list):
        log_proba = h[position*q + amino]
        for i in range(N):
            log_proba += 0.5*(J[i*q + seq[i], position*q + amino] + J[position*q + amino, i*q + seq[i]])
        proba_vec[k] = np.exp(log_proba/T)
    return normalize(proba_vec)

def accessible_states(old_codon):
    #given a codon returns the list of amino acids reachable from that codon with one DNA mutation
    old_codon = np.array([lettr for lettr in old_codon])
    codon_list = []
    for i in range(3):
        mutated_cod = np.copy(old_codon)
        for lettr in "ACGT":
            mutated_cod[i] = lettr
            if cod2amino("".join(mutated_cod)) != 20 :
                codon_list.append("".join(mutated_cod) )
    amino_list = list(set(map(cod2amino, codon_list)))
    return codon_list, amino_list
        
def evol_seq(seq, MC_steps, h, J, N, T = 1):
    #given a sequence returns an "evolved" sequence, that is a sequence with mutations introduced with Gibbs sampling
    mutated_seq = deepcopy(seq)
    non_gapped_pos = np.array([pos for pos, amino in enumerate(seq.Amino) if amino != 20])
    for steps in range(MC_steps):
        pos_mut = choice(non_gapped_pos)
        old_codon = mutated_seq.Codon[pos_mut]
        codon_list, amino_list = accessible_states(old_codon)
        cond_proba = cond_prob(pos_mut, amino_list, mutated_seq.Amino, h, J, N, T)
        new_amino = choice(amino_list, 1, p = cond_proba)[0]
        new_codon = choice([codon for codon in codon_list if cod2amino(codon) == new_amino])
        mutate_codon(mutated_seq, new_codon, pos_mut)
        mutate_amino(mutated_seq, new_amino, pos_mut)
    return mutated_seq

def evol_seq_sym(seq, MC_steps, h, J, N, T = 1):
    #symmetrized couplings version
    mutated_seq = deepcopy(seq)
    non_gapped_pos = np.array([pos for pos, amino in enumerate(seq.Amino) if amino != 20])
    for steps in range(MC_steps):
        pos_mut = choice(non_gapped_pos)
        old_codon = mutated_seq.Codon[pos_mut]
        codon_list, amino_list = accessible_states(old_codon)
        cond_proba = cond_prob_sym(pos_mut, amino_list, mutated_seq.Amino, h, J, N, T)
        new_amino = choice(amino_list, 1, p = cond_proba)[0]
        new_codon = choice([codon for codon in codon_list if cod2amino(codon) == new_amino])
        mutate_codon(mutated_seq, new_codon, pos_mut)
        mutate_amino(mutated_seq, new_amino, pos_mut)
    return mutated_seq

def evol_seq_profile(seq, MC_steps, p_prof, N, T = 1):
    #profile model version
    mutated_seq = deepcopy(seq)
    non_gapped_pos = np.array([pos for pos, amino in enumerate(seq.Amino) if amino != 20])
    for steps in range(MC_steps):
        pos_mut = choice(non_gapped_pos)
        old_codon = mutated_seq.Codon[pos_mut]
        codon_list, amino_list = accessible_states(old_codon)
        prob = p_prof[pos_mut*21 : pos_mut*21 + 21 ][amino_list]
        prob /= sum(prob)
        new_amino = choice(amino_list, 1, p = prob)[0]
        new_codon = choice([codon for codon in codon_list if cod2amino(codon) == new_amino])
        mutate_codon(mutated_seq, new_codon, pos_mut)
        mutate_amino(mutated_seq, new_amino, pos_mut)
    return mutated_seq
  
    
def energy(h, J, seq):
    #energy of a sequence with symmetrized couplings
    en = 0
    L = len(seq)
    for i in range(L):
        en -= h[i*q+seq[i]]
        for j in range(i):
            en -= 0.5*(J[j*q+seq[j],i*q+seq[i]] + J[i*q+seq[i],j*q+seq[j]])
    return en

In [ ]:
def plot_mut_spectrum(freqs, seq_amino, title, ref = "TEM", data = "freq"): 
    #to plot MSA mutational spectrum
    #-------USEFUL
    
    # fix the sides
    pos1 = 1
    pos2 = len(seq_amino)
        
    # length of the sequence to be displayed
    l = pos2 - pos1 + 1
    
    # sequence to display
    seq_amino = seq_amino[(pos1 -1): (pos2)]
    
    # sequence with None instead of gaps
    seq_amino_none_gap = [None if val == 20 else val for val in seq_amino]
    
    # add None in the positions where the wt has a gap
    freqs_nan = np.copy(freqs)
    for i, val in enumerate(seq_amino_none_gap):
        if val == None:
            freqs_nan[:, i] = [None for i in range(20)]

    #-------COLORS
    
    # define colors
    if data == "freq":
        norm1 = matplotlib.colors.LogNorm(vmin = 10^(-3), vmax = 1)
        labb = "Frequency" 
        ccc = ["white",  "#6a72da", "navy",  "limegreen", "limegreen"]
        cvals = [0, 0.16, 0.3, 0.5, 1] 
    elif data == "en":
        ccc = ["black", "green", "limegreen", "white","white", "#6a72da", "darkblue"]
        norm1 = norm1 = matplotlib.colors.Normalize(-6, 10)
        labb = "ΔE"
        cvals = map(norm1, [-6, -5, -1,  1.5, 4.5 , 7, 10])     
    elif data == "fit":
        ccc = ["darkblue", "#2932a6", "white", "limegreen",  "green"]
        norm1 = matplotlib.colors.LogNorm( 0.01, 2.5 )
        labb = "Fitness"
        cvals = map(norm1, [0.01, 0.1, 1.6, 2, 2.5]) 
    
    tuples =list(zip(cvals, ccc))
    cmap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", tuples, N = 50)
        
    # useful stuff
    bar_dict = dict([ ("shrink", 1.2), ("orientation", "vertical")])

    #-------MAINFIG
    
    # start figure
    fig1 = plt.figure(figsize=(25,3), dpi = 300)
    dat =  freqs_nan[:, range(pos1-1, pos2)]
    ax = sns.heatmap(dat, cmap = cmap1,  norm = norm1,
        square = True, center = 0.9,  linewidths=0.01, linecolor = "lightgrey", 
        cbar= True, cbar_kws = bar_dict)
    # add title
    plt.title(title, fontsize=20, y=1.08)
    
    # add labels to axis
    plt.ylabel("Amino acid",rotation=90,fontsize=20)
    plt.xlabel("Protein sequence position",rotation=0,fontsize=20) 

    # make frame visible
    for key in ["top", "bottom", "left", "right"]:
        ax.spines[key].set_visible(True)
        
    #-------COLORBAR
    
    # colorbal labels
    cbar = ax.collections[0].colorbar
    if data != "en":
        cbar.set_ticks([0, 10**(-3), 10**(-2), 10**(-1),0.5, 1, 2])
        cbar.set_ticklabels([0, "10⁻³", "10⁻²", "10⁻¹", 0.5, 1, 2])      
    else:
        cbar.set_ticks(range(-10, 10 + 1, 2))
        cbar.set_ticklabels(range(-10, 10 + 1, 2)) 
    cbar.set_label(labb,fontsize = 18)
    cbar.ax.tick_params(labelsize=18)
    ax.set_facecolor("grey")
    cbar.outline.set_visible(True)
    cbar.outline.set_edgecolor("black")
    cbar.outline.set_linewidth(1)
    axbar = cbar.ax
    axbar.set_aspect(13)
    
    #-------WT
    
    # add red scatter for wt
    amino_pos = [None if amino == None else amino + 0.5 for amino in seq_amino_none_gap]
    ok_pos = [None if val == None else i + 0.5 for i, val in enumerate(seq_amino_none_gap)]
    p = plt.scatter(ok_pos, amino_pos, color = "red", s = 5, alpha = 0.8, 
        linewidth = 0.0, edgecolors = "red")
    
    # add wt name  
    pos_leg = 1.35
    
    if ref == "AAC6":
        pos_leg = 1.25
    lgn = p.axes.legend([f"{ref}"] ,
        bbox_to_anchor = (1,pos_leg), 
    frameon = True, framealpha = 0, edgecolor = "black", fontsize = 15)
    lgn.legendHandles[0]._sizes = [130]
  
        
     #-------XLABELS
    
    # xlabels and xticks
    xxx_pos = [i for i in np.arange( 0.5, l - 0.5) ] 
    labs = [i for i in range(pos1, pos2+1)]
    ax.set_xticks(xxx_pos)
    ax.set_xticklabels(labels = labs, rotation=0, horizontalalignment="right", 
        color="k",fontsize= 12, ha = "center")
    
    # set xtiks every 10
    temp = ax.xaxis.get_major_ticks()
    for i, label in enumerate(temp): 
        if (i + 10)%10 != 0: 
            label.set_visible(False)
            
            
    #-------YLABELS
    
    # set ylabels to nothing
    ax.set_yticks([] )

    # add ylabel
    pos_amino_letts = 208
    amino_dists = 1.8
    amino_start = -8
    if ref == "AAC6":
        amino_dists = 1.2
    if ref == "AAC6":
        pos_amino_letts = 121
    if ref == "AAC6":
        amino_start = -2
    yyy = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L",  "M",  "N", "P",  "Q",  "R",
    "S",  "T", "V",  "W",  "Y"]
    for i in range(20):
        pp = amino_start + amino_dists*i + 1.5
        ax.text(pos_amino_letts, pp, yyy[i], fontsize = 12)
        
    # draw one black bar
    ax2 = plt.axes([0.745, 0.75, 0.018, 0.2])
    x_values = [ 0, 1]
    y_values = [0, 3]
    ax2.plot(x_values, y_values, color = "black")
    ax2.axis("off")
    
    # draw the other black bar
    ax3 = plt.axes([0.745, 0.05, 0.018, 0.2])
    x_values = [ 0, 1]
    y_values = [0, -3]
    ax3.plot(x_values, y_values, color = "black")
    ax3.axis("off")
    return fig1

## Data

Read the aligned DNA sequence of PSE-1.

In [ ]:
data_wt_DNA = open('./../data/PSE-1_DNA.fasta', 'r')
PSE1_DNA_string = data_wt_DNA.readlines()[1][:-1]
PSE1_codon = [PSE1_DNA_string[ (i*3) : ((i+1)*3) ] for i in range(L)]
data_wt_DNA.close()

Here is how it looks like.

In [ ]:
PSE1_codon[1:10]

Let's translate it into amino acids.. or numbers in our case.

In [ ]:
PSE1_amino = [cod2amino(PSE1_codon[i]) for i in range(L)]

In [ ]:
PSE1_amino[1:10]

<div class="alert alert-block alert-info">
<b>Note : </b>the genetic code is redundant! </div>

```python 
PSE1_codon[5] = 'TAT'
PSE1_codon[5] = 'TAC'
``` 

but 

```python 
PSE1_amino[5] = 19 aka 'Y'
PSE1_amino[5] = 19 aka 'Y'
``` 
this plays a non trivial role in evolution!

Let's now embed the data in the class `Sequence` for easier management.


In [ ]:
PSE1 = Sequence(PSE1_amino, PSE1_codon)


The functions `mutate_codon(Sequence, new_codon, pos)` and `mutate_amino(Sequence, new_amino, pos)` allow to easely change codons and amino acids in one sequence.

Given a sequence the model allows us to compute a *score* or *energy* (for those more keen to the physics jargon). <br>The lower the score, the better the sequence! This holds true as far as we stay withing the range of scores of natural sequences, generating sequences with lower scores might not be so miningful..

In [ ]:
E_PSE1 = energy(h, J, PSE1.Amino)
print(E_PSE1)

Let's compute the energy of all natural sequences - according to the model we learned.

In [ ]:
E_nat = [energy(h, J, align[i, :]) for i in range(M)]

In [ ]:
xsize = 3
ysize = 3
fig = plt.figure(figsize = (xsize, ysize), dpi = 200)
plt.hist(E_nat)
plt.show()
;

## Evolution

In [ ]:
We are going to 

Looking at the data that comes from this [Stiffler et al.](https://www.sciencedirect.com/science/article/pii/S2405471219304284") paper

In [ ]:
MSA_exp = fasta2matrix("../data/PSE-1_Round20_Stiffler_10000.fasta")

Energy and Hamming distance of Stiffler's data.

In [ ]:
Mm = 100
rr = choice(range(10000), Mm)
H_exp = [dH(MSA_exp[i, :], PSE1.Amino) for i in rr]
energy_exp = [energy(h, J, MSA_exp[i, :]) for i in rr]
print(np.mean(energy_exp)  - E_PSE1)
np.mean(H_exp)

Generating an in silico alignment..

In [ ]:
M_silico = 1000
N = 202
MSA_silico = np.zeros((M_silico, N), dtype = int)
start_t = time.time()
for i in range(M_silico):
    MSA_silico[i, :] = evol_seq(PSE1, 75, h, J, N, T = 1.2).Amino
    if (i > 0) & (i%100 == 0):
        print(f"Sampled {i} sequences (out of {M_silico})")
final_t = time.time()
print("\nDone! Enjoy your results.")
print(f"The sampling has taken {round(final_t - start_t)} seconds in total (~ {round((final_t - start_t)/60)} minutes).")

Let's compute the correlation with the data.

In [ ]:
def mut_profile(ali):
    M, L = np.shape(ali)
    ali_1hot = onehot(ali)
    freq = np.sum(ali_1hot,0)/M
    return np.reshape(freq, (L, 21))

In [ ]:
print(M_silico)
pi_silico = mut_profile(MSA_silico)
ran = choice(range(10000), M_silico)
pi_exp = mut_profile(MSA_exp[ran, :])
sc.spearmanr(pi_exp.flatten(),pi_silico.flatten())

Let's see them in a nice plot.

In [ ]:
fr_tmp = (mut_profile(MSA_silico_best[1:100]).T)[0:20, :]
mmm = np.array([val for val in fr_tmp.flatten() if val != 0]).min()
fr_tmp += mmm

plot_mut_spectrum(fr_tmp, PSE1.Amino, "Silico bmDCA", ref = "PSE-1", data = "freq")
;


fr_tmp = (mut_profile(MSA_exp[1:1000]).T)[0:20, :]
mmm = np.array([val for val in fr_tmp.flatten() if val != 0]).min()
fr_tmp += mmm
plot_mut_spectrum(fr_tmp, PSE1.Amino, "Exp", ref = "PSE-1", data = "freq")
;

fr_tmp = (mut_profile(MSA_silico[1:1000]).T)[0:20, :]
mmm = np.array([val for val in fr_tmp.flatten() if val != 0]).min()
fr_tmp += mmm
plot_mut_spectrum(fr_tmp, PSE1.Amino, "Silico plmDCA", ref = "PSE-1", data = "freq")
;

fr_tmp = (mut_profile(MSA_silico_prof[1:1000]).T)[0:20, :]
mmm = np.array([val for val in fr_tmp.flatten() if val != 0]).min()
fr_tmp += mmm
plot_mut_spectrum(fr_tmp, PSE1.Amino, "Profile model", ref = "PSE-1", data = "freq")
;



In [ ]:
plt.scatter(np.log(mut_profile(MSA_silico_prof[1:1000]).flatten()),
            np.log(mut_profile(MSA_exp[1:1000]).flatten()))

In [ ]:
plt.scatter(np.log(mut_profile(MSA_silico_best[1:1000]).flatten()),
            np.log(mut_profile(MSA_exp[1:1000]).flatten()))

What happens with the profile model?

In [ ]:
N = 202
M_silico_prof = 1000
MSA_silico_prof = np.zeros((M_silico_prof, N), dtype = int)
for i in range(M_silico_prof):
        if i%100 == 0:
            print(i)
        MSA_silico_prof[i, :] = evol_seq_profile(PSE1, 62, pa, N, T = 1).Amino


In [ ]:
print(M_silico_prof)
ran = choice(range(165000), M_silico_prof)
pi_prof = mut_profile(MSA_silico_prof)
pi_exp = mut_profile(MSA_exp[ran, :])
sc.spearmanr(pi_prof.flatten(),pi_exp.flatten())

What can we reach (in terms of correlation) with a better lerned model?

In [ ]:
MSA_silico_best = fasta2matrix("../data/PSE_silico_seqs_10000_T_1_4_MCsteps_75.fasta")

In [ ]:
M_sub = 1000
print(M_sub)
ran = choice(range(10000), M_sub)
pi_exp = mut_profile(MSA_exp[ran, :])
pi_silico_best = mut_profile(MSA_silico_best[ran, :])
sc.spearmanr(pi_silico_best.flatten(),pi_exp.flatten())